In [1]:
import tensorflow as tf
import numpy as np

In [2]:
np.random.seed(1)

# log dir
logs_path = '/tmp/logistic/3'

# reset everything to rerun in jupyter
tf.reset_default_graph()

In [3]:
learning_rate = 0.1
num_iters = 2000

In [4]:
X = np.array([[0, 1], [1, 1], [1, 0], [0, 0]])
y = np.array([0, 1, 0, 0], ndmin=2).T

In [5]:
with tf.name_scope("input"):
    train_X = tf.placeholder(tf.float32, name="train_X")
    train_y = tf.placeholder(tf.float32, name="train_y")

In [6]:
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([X.shape[1], 1]), dtype=tf.float32)
    
with tf.name_scope("bias"):
    b = tf.Variable(np.random.randn(), dtype=tf.float32)

In [7]:
with tf.name_scope("model"):
    pred = tf.nn.sigmoid(tf.matmul(train_X, W) + b)

In [8]:
with tf.name_scope("cost"):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(train_y * tf.log(pred), reduction_indices=1))

In [9]:
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [10]:
# create a summary for our cost and accuracy
tf.summary.scalar("cost", cross_entropy)

# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.summary.merge_all()

In [11]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        
    for epoch in np.arange(num_iters):
        _, summary = session.run([optimizer, summary_op], feed_dict={ train_X: X, train_y: y}) 
            
        # write log
        writer.add_summary(summary, epoch)

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({train_X: X, train_y: y}))

Accuracy: 1.0
